In [1]:
import requests
import pandas as pd
from datetime import datetime
import warnings
import numpy as np
from tqdm import tqdm
warnings.filterwarnings('ignore')

In [2]:
#URL = "https://tequila-api.kiwi.com/v2/search"  # Old url
URL = "https://api.tequila.kiwi.com/v2/search"

In [20]:
params = {
        'fly_from': None,
        'fly_to': None,
        'date_from': '11/07/2022',
        'date_to': '17/07/2022',
        'flight_type': 'oneway',
        'curr': 'INR',
        'one_for_city': 0,
        'one_per_date': 0,
        'adults': 1,
        'children': 0,
        'selected_cabins': None,
        'only_working_days': False,
        'only_weekends': False,
        'partner_market': 'us',
        'max_stopovers': 0,
        'vehicle_type': 'aircraft',
        'limit': 25
    }

API_KEY = <API_KEY>

In [21]:
# M: Economy,   W: Business,   C: First
seat_class_fare = {
    'M': 'economy_fare',
    'W': 'business_fare',
    'C': 'first_fare'
}
seat_class_code = {
    'economy': 'M',
    'business': 'W',
    'first': 'C'
}

In [28]:
def fetchData(origin, destination, seat_class, URL=URL, params=params):
    # M: Economy,   W: Business,   C: First
    params['fly_from'] = origin
    params['fly_to'] = destination
    params['selected_cabins'] = seat_class_code[seat_class]
    
    res = requests.get(URL, headers={"apikey":API_KEY}, params=params)
    return res.json()

In [23]:
def getdf(data):
    df = pd.DataFrame(columns=['origin','destination','depart_time','depart_weekday','duration','arrival_time','arrival_weekday','flight_no','airline_code','economy_fare','business_fare','first_fare'])
    for route in data['data']:
        d = datetime.strptime(route['route'][0]['local_departure'],"%Y-%m-%dT%H:%M:%S.%fZ")
        a = datetime.strptime(route['route'][0]['local_arrival'],"%Y-%m-%dT%H:%M:%S.%fZ")
        df = df.append({
            'origin': route['route'][0]['flyFrom'],
            'destination': route['route'][0]['flyTo'],
            'depart_time': str(d.time()),
            'depart_weekday': d.weekday(),
            'duration': str(a-d),
            'arrival_time': str(a.time()),
            'arrival_weekday': a.weekday(),
            'flight_no': f"{route['route'][0]['airline']}{route['route'][0]['flight_no']}",
            'airline_code': route['route'][0]['airline'],
            f'{seat_class_fare[route["route"][0]["fare_category"]]}': route['price']
        }, ignore_index=True)
    return df

In [24]:
def getFinalDataFrame(eco_data, buss_data, first_data):
    eco_df = getdf(eco_data)
    buss_df = getdf(buss_data)
    first_df = getdf(first_data)
    
    for idx, row in buss_df.iterrows():
        match = eco_df.loc[(eco_df['origin']==row['origin']) & (eco_df['destination']==row['destination']) & (eco_df['depart_time']==row['depart_time']) & (eco_df['depart_weekday']==row['depart_weekday']) & (eco_df['arrival_time']==row['arrival_time']) & (eco_df['arrival_weekday']==row['arrival_weekday']) & (eco_df['flight_no']==row['flight_no']) & (eco_df['airline_code']==row['airline_code'])]
        for i, r in match.iterrows():
            eco_df.loc[i, 'business_fare'] = row['business_fare']
    
    for idx, row in first_df.iterrows():
        match = eco_df.loc[(eco_df['origin']==row['origin']) & (eco_df['destination']==row['destination']) & (eco_df['depart_time']==row['depart_time']) & (eco_df['depart_weekday']==row['depart_weekday']) & (eco_df['arrival_time']==row['arrival_time']) & (eco_df['arrival_weekday']==row['arrival_weekday']) & (eco_df['flight_no']==row['flight_no']) & (eco_df['airline_code']==row['airline_code'])]
        for i, r in match.iterrows():
            eco_df.loc[i, 'first_fare'] = row['first_fare']
    
    return eco_df

In [29]:
# M: Economy,   W: Business,   C: First
eco_data = fetchData('DEL', 'HND', 'economy')
buss_data = fetchData('DEL', 'HND', 'business')
first_data = fetchData('DEL', 'HND', 'first')

In [33]:
## Testing New API Code
#params = {
#        'fly_from': 'DEL',
#        'fly_to': 'HND',
#        'date_from': '20/04/2023',
#        'date_to': '28/04/2023',
#        'flight_type': 'oneway',
#        'curr': 'INR',
#        'one_for_city': 0,
#        'one_per_date': 0,
#        'adults': 1,
#        'children': 0,
#        'selected_cabins': 'M',
#        'only_working_days': False,
#        'only_weekends': False,
#        'partner_market': 'us',
#        'max_stopovers': 0,
#        'vehicle_type': 'aircraft',
#        'limit': 25
#    }
#
#API_KEY = <API_KEY>
#URL = "https://api.tequila.kiwi.com/v2/search"
#
#res = requests.get(URL, headers={"apikey":API_KEY}, params=params)
#res.json()

{'search_id': '7eef2481-26f6-c1f7-e36e-f6c16454cb43',
 'currency': 'INR',
 'fx_rate': 90.046404,
 'data': [{'id': '0711097b4c1000000049b791_0',
   'flyFrom': 'DEL',
   'flyTo': 'HND',
   'cityFrom': 'New Delhi',
   'cityCodeFrom': 'DEL',
   'cityTo': 'Tokyo',
   'cityCodeTo': 'TYO',
   'countryFrom': {'code': 'IN', 'name': 'India'},
   'countryTo': {'code': 'JP', 'name': 'Japan'},
   'nightsInDest': None,
   'quality': 774.095769,
   'distance': 5866.18,
   'duration': {'departure': 29100, 'return': 0, 'total': 29100},
   'price': 62801,
   'conversion': {'EUR': 697.429294, 'INR': 62801},
   'bags_price': {'1': 0, '2': 0},
   'baglimit': {'hand_height': 40,
    'hand_length': 55,
    'hand_weight': 10,
    'hand_width': 20,
    'hold_dimensions_sum': 158,
    'hold_height': 52,
    'hold_length': 78,
    'hold_weight': 23,
    'hold_width': 28},
   'availability': {'seats': 9},
   'airlines': ['NH'],
   'route': [{'id': '0711097b4c1000000049b791_0',
     'combination_id': '0711097b4c10

In [9]:
df = getFinalDataFrame(eco_data, buss_data, first_data)
df

,origin,destination,depart_time,depart_weekday,duration,arrival_time,arrival_weekday,flight_no,airline_code,economy_fare,business_fare,first_fare
0,DEL,HND,18:00:00,0,11:55:00,05:55:00,1,NH838,NH,64969,118058,NaN
1,DEL,HND,18:00:00,6,11:55:00,05:55:00,0,NH838,NH,64969,NaN,145266
2,DEL,HND,18:00:00,2,11:55:00,05:55:00,3,NH838,NH,84597,91316,NaN
3,DEL,HND,18:00:00,4,11:55:00,05:55:00,5,NH838,NH,84597,NaN,NaN
4,DEL,HND,19:05:00,2,11:50:00,06:55:00,3,JL30,JL,90755,NaN,181934
5,DEL,HND,19:05:00,4,11:50:00,06:55:00,5,JL30,JL,90755,NaN,NaN
6,DEL,HND,19:05:00,6,11:50:00,06:55:00,0,JL30,JL,90755,NaN,181934


In [10]:
pdf = pd.read_csv("places.csv")
pdf.head()

,city,airport,code,country
0,Atlanta,Hartsfield–Jackson Atlanta International Airport,ATL,United States
1,Beijing,Beijing Capital International Airport,PEK,China
2,Dubai,Dubai International Airport,DXB,United Arab Emirates
3,Los Angeles,Los Angeles International Airport,LAX,United States
4,Chicago,O'Hare International Airport,ORD,United States


In [27]:
cities = list(pdf['code'][:30])
len(cities)

30

In [28]:
cities[:5]

['ATL', 'PEK', 'DXB', 'LAX', 'ORD']

In [29]:
final_df = None

In [30]:
for i in tqdm(cities):
    for j in cities:
        if i!=j:
            print(i,"->",j+": ", end='')
            eco_data = fetchData(i, j, 'economy')
            buss_data = fetchData(i, j, 'business')
            first_data = fetchData(i, j, 'first')
            df = getFinalDataFrame(eco_data, buss_data, first_data)
            if final_df is None:
                final_df = df
            else:
                final_df = final_df.append(df, ignore_index=True).reset_index(drop=True)
            print("DONE")

  0%|                                                                                           | 0/30 [00:00<?, ?it/s]

ATL -> PEK: DONE
ATL -> DXB: DONE
ATL -> LAX: DONE
ATL -> ORD: DONE
ATL -> LHR: DONE
ATL -> HND: DONE
ATL -> HKG: DONE
ATL -> PVG: DONE
ATL -> CDG: DONE
ATL -> AMS: DONE
ATL -> DFW: DONE
ATL -> CAN: DONE
ATL -> FRA: DONE
ATL -> IST: DONE
ATL -> DEL: DONE
ATL -> CGK: DONE
ATL -> SIN: DONE
ATL -> ICN: DONE
ATL -> DEN: DONE
ATL -> BKK: DONE
ATL -> JFK: DONE
ATL -> KUL: DONE
ATL -> SFO: DONE
ATL -> MAD: DONE
ATL -> CTU: DONE
ATL -> LAS: DONE
ATL -> BCN: DONE
ATL -> BOM: DONE
ATL -> YYZ: 

  3%|██▋                                                                             | 1/30 [03:34<1:43:51, 214.87s/it]

DONE
PEK -> ATL: DONE
PEK -> DXB: DONE
PEK -> LAX: DONE
PEK -> ORD: DONE
PEK -> LHR: DONE
PEK -> HND: DONE
PEK -> HKG: DONE
PEK -> PVG: DONE
PEK -> CDG: DONE
PEK -> AMS: DONE
PEK -> DFW: DONE
PEK -> CAN: DONE
PEK -> FRA: DONE
PEK -> IST: DONE
PEK -> DEL: DONE
PEK -> CGK: DONE
PEK -> SIN: DONE
PEK -> ICN: DONE
PEK -> DEN: DONE
PEK -> BKK: DONE
PEK -> JFK: DONE
PEK -> KUL: DONE
PEK -> SFO: DONE
PEK -> MAD: DONE
PEK -> CTU: DONE
PEK -> LAS: DONE
PEK -> BCN: DONE
PEK -> BOM: DONE
PEK -> YYZ: 

  7%|█████▎                                                                          | 2/30 [04:46<1:00:49, 130.32s/it]

...

DONE
BOM -> ATL: DONE
BOM -> PEK: DONE
BOM -> DXB: DONE
BOM -> LAX: DONE
BOM -> ORD: DONE
BOM -> LHR: DONE
BOM -> HND: DONE
BOM -> HKG: DONE
BOM -> PVG: DONE
BOM -> CDG: DONE
BOM -> AMS: DONE
BOM -> DFW: DONE
BOM -> CAN: DONE
BOM -> FRA: DONE
BOM -> IST: DONE
BOM -> DEL: DONE
BOM -> CGK: DONE
BOM -> SIN: DONE
BOM -> ICN: DONE
BOM -> DEN: DONE
BOM -> BKK: DONE
BOM -> JFK: DONE
BOM -> KUL: DONE
BOM -> SFO: DONE
BOM -> MAD: DONE
BOM -> CTU: DONE
BOM -> LAS: DONE
BOM -> BCN: DONE
BOM -> YYZ: 

 97%|████████████████████████████████████████████████████████████████████████████▎  | 29/30 [1:25:12<02:18, 138.59s/it]

DONE
YYZ -> ATL: DONE
YYZ -> PEK: DONE
YYZ -> DXB: DONE
YYZ -> LAX: DONE
YYZ -> ORD: DONE
YYZ -> LHR: DONE
YYZ -> HND: DONE
YYZ -> HKG: DONE
YYZ -> PVG: DONE
YYZ -> CDG: DONE
YYZ -> AMS: DONE
YYZ -> DFW: DONE
YYZ -> CAN: DONE
YYZ -> FRA: DONE
YYZ -> IST: DONE
YYZ -> DEL: DONE
YYZ -> CGK: DONE
YYZ -> SIN: DONE
YYZ -> ICN: DONE
YYZ -> DEN: DONE
YYZ -> BKK: DONE
YYZ -> JFK: DONE
YYZ -> KUL: DONE
YYZ -> SFO: DONE
YYZ -> MAD: DONE
YYZ -> CTU: DONE
YYZ -> LAS: DONE
YYZ -> BCN: DONE
YYZ -> BOM: 

100%|███████████████████████████████████████████████████████████████████████████████| 30/30 [1:29:40<00:00, 179.34s/it]

DONE


In [31]:
final_df.tail()

,origin,destination,depart_time,depart_weekday,duration,arrival_time,arrival_weekday,flight_no,airline_code,economy_fare,business_fare,first_fare
7804,YYZ,BCN,20:10:00,5,13:55:00,10:05:00,6,LH6635,LH,91759,NaN,466497
7805,YYZ,BCN,20:10:00,3,13:55:00,10:05:00,4,LH6635,LH,93052,NaN,NaN
7806,YYZ,BCN,20:10:00,6,13:55:00,10:05:00,0,LH6635,LH,93052,NaN,466496
7807,YYZ,BCN,20:10:00,4,13:55:00,10:05:00,5,LH6635,LH,93052,NaN,NaN
7808,YYZ,BCN,20:10:00,0,13:55:00,10:05:00,1,LH6635,LH,93052,111320,NaN


In [32]:
final_df.to_csv("international_flights.csv")